In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from eipy.ei import EnsembleIntegration
import eipy.utils
from eipy.additional_ensembles import MeanAggregation, CES
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import datasets

In [2]:
# If data is multi-class, run a check on the allowable base and meta models.

base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(multi_class="multinomial"),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}

In [3]:

"""
For filtering base predictors by whether or not they rely on heursitics for multiclass extension

natively_multi_class_predictors = ["XGBClassifier",
"BernoulliNB",
"DecisionTreeClassifier",
"ExtraTreeClassifier",
"GaussianNB",
"KNeighborsClassifier",
"LabelPropagation",
"LabelSpreading",
"LinearDiscriminantAnalysis",
"LinearSVC", #(setting multi_class=”crammer_singer”)
"LogisticRegression", #(setting multi_class=”multinomial”)
"LogisticRegressionCV", #(setting multi_class=”multinomial”)
"MLPClassifier",
"NearestCentroid",
"QuadraticDiscriminantAnalysis",
"RadiusNeighborsClassifier",
"RandomForestClassifier",
"RidgeClassifier",
"RidgeClassifierCV"]

base_predictors = {k : v for k,v in base_predictors.items() if str(v).split("(")[0] in natively_multi_class_predictors}
"""

'\nFor filtering base predictors by whether or not they rely on heursitics for multiclass extension\n\nnatively_multi_class_predictors = ["XGBClassifier",\n"BernoulliNB",\n"DecisionTreeClassifier",\n"ExtraTreeClassifier",\n"GaussianNB",\n"KNeighborsClassifier",\n"LabelPropagation",\n"LabelSpreading",\n"LinearDiscriminantAnalysis",\n"LinearSVC", #(setting multi_class=”crammer_singer”)\n"LogisticRegression", #(setting multi_class=”multinomial”)\n"LogisticRegressionCV", #(setting multi_class=”multinomial”)\n"MLPClassifier",\n"NearestCentroid",\n"QuadraticDiscriminantAnalysis",\n"RadiusNeighborsClassifier",\n"RandomForestClassifier",\n"RidgeClassifier",\n"RidgeClassifierCV"]\n\nbase_predictors = {k : v for k,v in base_predictors.items() if str(v).split("(")[0] in natively_multi_class_predictors}\n'

In [4]:
"""https://dev.pages.lis-lab.fr/scikit-multimodallearn/tutorial/auto_examples/combo/plot_combo_3_views_3_classes.html#
multi modal multi-class toy data generation"""

def generate_data(n_samples, lim):
    """Generate random data in a rectangle"""
    lim = np.array(lim)
    n_features = lim.shape[0]
    data = np.random.random((n_samples, n_features))
    data = (lim[:, 1]-lim[:, 0]) * data + lim[:, 0]
    return data
seed = 12
np.random.seed(seed)

n_samples = 300

modality_0 = np.concatenate((generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]])))

modality_1 = np.concatenate((generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]])))

modality_2 = np.concatenate((generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]])))

y = np.zeros(3*n_samples, dtype=np.int64)
y[n_samples:2*n_samples] = 1
y[2*n_samples:] = 2


In [5]:
X_0_train, X_0_test, y_train, y_test = train_test_split(modality_0, y, test_size=0.2, random_state=3, stratify=y)
X_1_train, X_1_test, _,_ = train_test_split(modality_1, y, test_size=0.2, random_state=3, stratify=y)
X_2_train, X_2_test, _,_ = train_test_split(modality_2, y, test_size=0.2, random_state=3, stratify=y)

In [6]:
data_train = {
                "Modality_0": X_0_train,
                "Modality_1": X_1_train,
                "Modality_2": X_2_train
                }

data_test = {
                "Modality_0": X_0_test,
                "Modality_1": X_1_test,
                "Modality_2": X_2_test
                }

In [7]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,
                        n_jobs=-1,
                        random_state=42,
                        project_name="toy",
                        model_building=True,
                        )



In [8]:
for name, modality in data_train.items():
    EI.train_base(modality, y_train, modality_name=name)

Training base predictors on Modality_0...
        
... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on Modality_1...
        
... for ensemble performance analysis...


...

Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on Modality_2...
        
... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


In [9]:
EI.meta_training_data[0]

modality          Modality_0                                              \
base predictor          ADAB                               XGB             
sample                     0                                 0             
class                      0             1         2         0         1   
0               5.033375e-01  2.222770e-16  0.496663  0.990476  0.000190   
1               5.033375e-01  2.222770e-16  0.496663  0.082868  0.000607   
2               2.223397e-16  5.036501e-01  0.496350  0.008628  0.492612   
3               2.223397e-16  5.036501e-01  0.496350  0.000314  0.914500   
4               5.033375e-01  2.222770e-16  0.496663  0.205049  0.002299   
..                       ...           ...       ...       ...       ...   
571             5.034012e-01  2.222858e-16  0.496599  0.730658  0.000891   
572             5.034012e-01  2.222858e-16  0.496599  0.741070  0.003562   
573             2.223192e-16  5.035152e-01  0.496485  0.001255  0.920323   
574             5.034012e-01  2.222858e-16  0.496599  0.988050  0.000777   
575             2.223192e-16  5.035152e-01  0.496485  0.000728  0.915664   

modality                                       ... Modality_2            \
base predictor             DT              RF  ...         NB             
sample                      0               0  ...          0             
class                  2    0    1    2     0  ...          0         1   
0               0.009334  1.0  0.0  0.0  0.91  ...   0.293706  0.014156   
1               0.916525  0.0  0.0  1.0  0.54  ...   0.271800  0.725416   
2               0.498759  0.0  0.0  1.0  0.04  ...   0.253442  0.746303   
3               0.085186  0.0  1.0  0.0  0.01  ...   0.230185  0.003556   
4               0.792652  1.0  0.0  0.0  0.52  ...   0.238629  0.002247   
..                   ...  ...  ...  ...   ...  ...        ...       ...   
571             0.268450  1.0  0.0  0.0  0.85  ...   0.319283  0.676623   
572             0.255368  0.0  0.0  1.0  0.73  ...   0.278899  0.721085   
573             0.078422  0.0  1.0  0.0  0.00  ...   0.222018  0.000400   
574             0.011173  1.0  0.0  0.0  0.87  ...   0.356938  0.632894   
575             0.083608  0.0  1.0  0.0  0.00  ...   0.241844  0.002259   

modality                                                                    \
base predictor                 MLP                           SVM             
sample                           0                             0             
class                  2         0         1         2         0         1   
0               0.692139  0.344770  0.058037  0.597193  0.235021  0.003676   
1               0.002784  0.356892  0.614702  0.028406  0.314243  0.683171   
2               0.000256  0.341900  0.651430  0.006671  0.335281  0.662332   
3               0.766259  0.322704  0.029860  0.647435  0.191561  0.002338   
4               0.759124  0.334455  0.027040  0.638505  0.235444  0.001804   
..                   ...       ...       ...       ...       ...       ...   
571             0.004094  0.366665  0.608913  0.024422  0.286633  0.711375   
572             0.000017  0.341974  0.653626  0.004401  0.324069  0.671813   
573             0.777582  0.285722  0.017523  0.696755  0.276874  0.003653   
574             0.010168  0.380057  0.586231  0.033712  0.292738  0.702961   
575             0.755897  0.324260  0.036452  0.639288  0.273303  0.004605   

modality                 labels  
base predictor                   
sample                           
class                  2         
0               0.761303      0  
1               0.002586      0  
2               0.002388      1  
3               0.806101      2  
4               0.762752      0  
..                   ...    ...  
571             0.001992      0  
572             0.004119      0  
573             0.719473      2  
574             0.004301      0  
575             0.722092      2  

[576 rows x 91 columns]

In [10]:
EI.meta_test_data[0]

modality          Modality_0                                              \
base predictor          ADAB                               XGB             
sample                     0                                 0             
class                      0             1         2         0         1   
0               2.223189e-16  5.035294e-01  0.496471  0.000618  0.720623   
1               5.034139e-01  2.222883e-16  0.496586  0.219468  0.005430   
2               5.034139e-01  2.222883e-16  0.496586  0.732608  0.000160   
3               5.034139e-01  2.222883e-16  0.496586  0.876348  0.000254   
4               5.034139e-01  2.222883e-16  0.496586  0.492079  0.000627   
..                       ...           ...       ...       ...       ...   
139             5.034139e-01  2.222883e-16  0.496586  0.598946  0.001246   
140             5.034139e-01  2.222883e-16  0.496586  0.895722  0.000379   
141             2.223189e-16  5.035294e-01  0.496471  0.000606  0.628569   
142             2.223189e-16  5.035294e-01  0.496471  0.000275  0.901953   
143             2.223189e-16  5.035294e-01  0.496471  0.000562  0.853588   

modality                                       ... Modality_2            \
base predictor             DT              RF  ...         NB             
sample                      0               0  ...          0             
class                  2    0    1    2     0  ...          0         1   
0               0.278758  0.0  1.0  0.0  0.00  ...   0.412818  0.065808   
1               0.775102  0.0  0.0  1.0  0.24  ...   0.254391  0.000053   
2               0.267231  1.0  0.0  0.0  0.70  ...   0.267024  0.000076   
3               0.123398  1.0  0.0  0.0  0.76  ...   0.468849  0.142448   
4               0.507294  0.0  0.0  1.0  0.63  ...   0.204523  0.001023   
..                   ...  ...  ...  ...   ...  ...        ...       ...   
139             0.399808  1.0  0.0  0.0  0.65  ...   0.377372  0.050351   
140             0.103899  1.0  0.0  0.0  0.72  ...   0.222622  0.000360   
141             0.370825  0.0  1.0  0.0  0.00  ...   0.392514  0.051587   
142             0.097772  0.0  1.0  0.0  0.01  ...   0.250095  0.749646   
143             0.145850  0.0  1.0  0.0  0.00  ...   0.479250  0.149968   

modality                                                                    \
base predictor                 MLP                           SVM             
sample                           0                             0             
class                  2         0         1         2         0         1   
0               0.521374  0.349900  0.098474  0.551626  0.398528  0.018970   
1               0.745556  0.272021  0.003683  0.724295  0.283650  0.006901   
2               0.732901  0.293355  0.004577  0.702068  0.307633  0.001984   
3               0.388703  0.393194  0.156214  0.450592  0.555346  0.069046   
4               0.794454  0.285422  0.012241  0.702337  0.232020  0.002750   
..                   ...       ...       ...       ...       ...       ...   
139             0.572277  0.318369  0.071767  0.609864  0.376655  0.014596   
140             0.777019  0.293802  0.008447  0.697751  0.282284  0.001291   
141             0.555899  0.337723  0.083950  0.578327  0.363116  0.013678   
142             0.000259  0.341602  0.652026  0.006372  0.323515  0.675633   
143             0.370782  0.400836  0.172430  0.426734  0.552639  0.075290   

modality                 labels  
base predictor                   
sample                           
class                  2         
0               0.582502      2  
1               0.709450      2  
2               0.690382      0  
3               0.375608      2  
4               0.765230      0  
..                   ...    ...  
139             0.608749      0  
140             0.716424      0  
141             0.623206      2  
142             0.000853      1  
143             0.372071      2  

[144 rows x 91 columns]

In [32]:
columns_for_eval = [col[:-1] for col in EI.meta_test_data[0].columns]
columns_for_eval

[('Modality_0', 'ADAB', 0),
 ('Modality_0', 'ADAB', 0),
 ('Modality_0', 'ADAB', 0),
 ('Modality_0', 'XGB', 0),
 ('Modality_0', 'XGB', 0),
 ('Modality_0', 'XGB', 0),
 ('Modality_0', 'DT', 0),
 ('Modality_0', 'DT', 0),
 ('Modality_0', 'DT', 0),
 ('Modality_0', 'RF', 0),
 ('Modality_0', 'RF', 0),
 ('Modality_0', 'RF', 0),
 ('Modality_0', 'GB', 0),
 ('Modality_0', 'GB', 0),
 ('Modality_0', 'GB', 0),
 ('Modality_0', 'KNN', 0),
 ('Modality_0', 'KNN', 0),
 ('Modality_0', 'KNN', 0),
 ('Modality_0', 'LR', 0),
 ('Modality_0', 'LR', 0),
 ('Modality_0', 'LR', 0),
 ('Modality_0', 'NB', 0),
 ('Modality_0', 'NB', 0),
 ('Modality_0', 'NB', 0),
 ('Modality_0', 'MLP', 0),
 ('Modality_0', 'MLP', 0),
 ('Modality_0', 'MLP', 0),
 ('Modality_0', 'SVM', 0),
 ('Modality_0', 'SVM', 0),
 ('Modality_0', 'SVM', 0),
 ('Modality_1', 'ADAB', 0),
 ('Modality_1', 'ADAB', 0),
 ('Modality_1', 'ADAB', 0),
 ('Modality_1', 'XGB', 0),
 ('Modality_1', 'XGB', 0),
 ('Modality_1', 'XGB', 0),
 ('Modality_1', 'DT', 0),
 ('Modality

In [34]:
for column in columns_for_eval:
        EI.meta_test_data[0][column]

0      2.223189e-16
1      5.034139e-01
2      5.034139e-01
3      5.034139e-01
4      5.034139e-01
           ...     
139    5.034139e-01
140    5.034139e-01
141    2.223189e-16
142    2.223189e-16
143    2.223189e-16
Name: 0, Length: 144, dtype: float64
0      2.223189e-16
1      5.034139e-01
2      5.034139e-01
3      5.034139e-01
4      5.034139e-01
           ...     
139    5.034139e-01
140    5.034139e-01
141    2.223189e-16
142    2.223189e-16
143    2.223189e-16
Name: 0, Length: 144, dtype: float64
0      2.223189e-16
1      5.034139e-01
2      5.034139e-01
3      5.034139e-01
4      5.034139e-01
           ...     
139    5.034139e-01
140    5.034139e-01
141    2.223189e-16
142    2.223189e-16
143    2.223189e-16
Name: 0, Length: 144, dtype: float64
0      0.000618
1      0.219468
2      0.732608
3      0.876348
4      0.492079
         ...   
139    0.598946
140    0.895722
141    0.000606
142    0.000275
143    0.000562
Name: 0, Length: 144, dtype: float32
0      0.000618


/tmp/ipykernel_611983/2735563482.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  print(EI.meta_test_data[0][column][0])


In [12]:
for frame in EI.meta_test_data:
    for modality in frame.columns.levels[0]:
        for model in frame.columns.levels[1]:
            for sample in frame.columns.levels[2]:
                frame[modality,model,sample] = np.argmax(frame[modality,model,sample])
EI.meta_test_data[0]

/tmp/ipykernel_611983/4200397984.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  frame[modality,model,sample] = np.argmax(frame[modality,model,sample])


KeyError: ('Modality_0', '', 0)

In [ ]:
EI.base_summary

{'metrics': modality       Modality_0                                                    \
 base predictor       ADAB        DT        GB       KNN        LR       MLP   
 precision        0.111111  0.111111  0.111111  0.111111  0.111111  0.111111   
 recall           0.333333  0.333333  0.333333  0.333333  0.333333  0.333333   
 f1               0.166667  0.166667  0.166667  0.166667  0.166667  0.166667   
 
 modality                                                ... Modality_2  \
 base predictor        NB        RF       SVM       XGB  ...       ADAB   
 precision       0.111111  0.111111  0.111111  0.111111  ...   0.111111   
 recall          0.333333  0.333333  0.333333  0.333333  ...   0.333333   
 f1              0.166667  0.166667  0.166667  0.166667  ...   0.166667   
 
 modality                                                                    \
 base predictor        DT        GB       KNN        LR       MLP        NB   
 precision       0.111111  0.111111  0.111111  0.111

In [ ]:
EI.train_meta(meta_predictors=base_predictors)

Analyzing ensembles: |          |  0%

Analyzing ensembles: |          |  0%


TypeError: type numpy.ndarray doesn't define __round__ method

In [ ]:
EI.meta_summary["metrics"]

,ADAB,XGB,DT,RF,GB,KNN,LR,NB,MLP,SVM
precision,0.926064,0.997245,0.995833,0.994473,0.995885,0.987952,0.989153,0.995839,0.989147,0.994496
recall,0.919444,0.997222,0.995833,0.994444,0.995833,0.987500,0.988889,0.995833,0.988889,0.994444
f1,0.919244,0.997222,0.995830,0.994450,0.995833,0.987555,0.988911,0.995833,0.988914,0.994444


In [ ]:
y_pred = EI.predict(X_dict=data_test, meta_model_key="XGB")
y_pred = np.round(y_pred)
y_pred

array([2., 1., 2., 0., 0., 2., 1., 2., 0., 2., 1., 1., 2., 1., 0., 1., 1.,
       2., 1., 1., 0., 0., 2., 0., 1., 1., 1., 1., 2., 0., 2., 2., 0., 2.,
       0., 0., 2., 1., 0., 1., 0., 0., 2., 1., 1., 2., 1., 0., 2., 2., 1.,
       1., 0., 0., 1., 1., 1., 1., 2., 2., 1., 0., 0., 2., 0., 2., 0., 1.,
       0., 1., 1., 2., 2., 0., 1., 2., 1., 1., 2., 0., 0., 0., 2., 1., 0.,
       0., 1., 1., 2., 0., 0., 2., 0., 0., 0., 1., 1., 2., 2., 1., 1., 2.,
       1., 1., 2., 2., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
       2., 2., 1., 1., 0., 0., 0., 2., 0., 1., 1., 2., 2., 2., 2., 2., 0.,
       2., 0., 1., 1., 2., 0., 1., 0., 2., 2., 2., 2., 1., 0., 0., 2., 1.,
       1., 2., 1., 1., 2., 0., 2., 0., 1., 0., 0., 2., 2., 0., 2., 2., 2.,
       0., 0., 0., 2., 1., 2., 1., 2., 0., 1.])

In [ ]:
y_test

array([2, 1, 2, 0, 0, 2, 1, 2, 0, 2, 1, 1, 2, 1, 0, 1, 1, 2, 1, 1, 0, 0,
       2, 0, 1, 1, 1, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 1, 0, 1, 0, 0, 2, 1,
       1, 2, 1, 0, 2, 2, 1, 1, 0, 0, 1, 1, 1, 1, 2, 2, 1, 0, 0, 2, 0, 2,
       0, 1, 0, 1, 1, 2, 2, 0, 1, 2, 1, 1, 2, 0, 0, 0, 2, 1, 0, 0, 1, 1,
       2, 0, 0, 2, 0, 0, 0, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 2, 1, 1, 0, 0, 0, 2, 0, 1, 1, 2, 2,
       2, 2, 2, 0, 2, 0, 1, 1, 2, 0, 1, 0, 2, 2, 2, 2, 1, 0, 0, 2, 1, 1,
       2, 1, 1, 2, 0, 2, 0, 1, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 1, 2,
       1, 2, 0, 1])

In [ ]:
accuracy = sum([1*(y==y_hat)+0*(y!=y_hat) for y,y_hat in list(zip(y_test, y_pred))])/len(y_test)
accuracy # =179/180

0.9944444444444445

In [ ]:
EI.meta_summary['thresholds']["XGB"]

precision    0.997222
recall       0.997222
f1           0.997222
Name: XGB, dtype: float64

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

Modality_a = X[:, 0:2]
Modality_b = X[:, 2:4]

X_a_train, X_a_test, y_train, y_test = train_test_split(Modality_a, y, test_size=0.2, random_state=3, stratify=y)
X_b_train, X_b_test, _,_ = train_test_split(Modality_b, y, test_size=0.2, random_state=3, stratify=y)

In [ ]:
iris_data_train = {
                "Modality_a": X_a_train,
                "Modality_b": X_b_train
                }

iris_data_test = {
                "Modality_a": X_a_test,
                "Modality_b": X_b_test
                }

In [ ]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}

In [ ]:
EI_iris = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,
                        n_jobs=-1,
                        random_state=0,
                        project_name="iris",
                        model_building=True,
                        )


In [ ]:
for name, modality in iris_data_train.items():
    EI_iris.train_base(modality, y_train, modality_name=name)

Training base predictors on Modality_a...
        
... for ensemble performance analysis...


Generating meta training data: |          |  0%

Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on Modality_b...
        
... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


In [ ]:
EI_iris.meta_training_data

[modality       Modality_a                                 ... Modality_b     \
 base predictor       ADAB XGB DT RF GB KNN LR NB MLP SVM  ...        XGB DT   
 sample                  0   0  0  0  0   0  0  0   0   0  ...          0  0   
 0                       1   1  1  1  1   0  1  1   1   1  ...          1  1   
 1                       2   1  1  1  1   1  1  1   2   1  ...          1  1   
 2                       2   2  2  2  2   2  2  2   2   2  ...          1  1   
 3                       2   2  2  2  2   2  2  2   2   2  ...          2  2   
 4                       1   0  0  0  0   0  0  0   0   0  ...          0  0   
 ..                    ...  .. .. .. ..  .. .. ..  ..  ..  ...        ... ..   
 91                      2   2  2  2  2   2  1  1   2   1  ...          2  2   
 92                      1   1  2  1  1   1  1  1   2   1  ...          1  1   
 93                      0   0  0  0  0   0  0  0   0   0  ...          0  0   
 94                      0   0  0  0  0 

In [ ]:
EI_iris.train_meta(meta_predictors=base_predictors)

Analyzing ensembles: |          |  0%

Analyzing ensembles: |██████████|100%
Training final meta models: |██████████|100%


In [ ]:
EI_iris.meta_summary["metrics"]

,ADAB,XGB,DT,RF,GB,KNN,LR,NB,MLP,SVM
precision,0.897778,0.974990,0.966667,0.983333,0.975193,0.975193,0.983333,0.936527,0.975193,0.983333
recall,0.883333,0.975000,0.966667,0.983333,0.975000,0.975000,0.983333,0.933333,0.975000,0.983333
f1,0.881481,0.974893,0.966667,0.983333,0.974996,0.974996,0.983333,0.933732,0.974996,0.983333


In [ ]:
y_pred_iris = EI_iris.predict(X_dict=iris_data_test, meta_model_key="SVM")
y_pred_iris = np.round(y_pred_iris)
y_pred_iris

array([2, 1, 2, 0, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 0, 2, 2, 0, 0, 0, 1, 0,
       0, 0, 2, 1, 0, 0, 1, 2])

In [ ]:
accuracy = sum([1*(y==y_hat)+0*(y!=y_hat) for y,y_hat in list(zip(y_test, y_pred_iris))])/len(y_test)
accuracy

0.9

In [ ]:
import pandas as pd
import numpy as np

# Create a dictionary with 2D arrays as values
data = {
    'Column1': [np.array([1, 2]), np.array([3, 4]), np.array([5, 6])],
    'Column2': [np.array([10, 20]), np.array([30, 40]), np.array([50, 60])]
}

# Create a DataFrame with MultiIndex
df = pd.DataFrame()

# Iterate over the dictionary items and reshape 2D arrays into sub-columns
for key, value in data.items():
    num_cols = value[0].shape[0]  # Get the shape of the 2D array
    print(num_cols)
    for i in range(num_cols):
        # Create sub-columns for each element of the 2D array
        sub_column_name = f'SubColumn{i + 1}'
        df[(key, sub_column_name)] = [row[i] for row in value]

# Print the DataFrame
print(df)


2
2
   (Column1, SubColumn1)  (Column1, SubColumn2)  (Column2, SubColumn1)  \
0                      1                      2                     10   
1                      3                      4                     30   
2                      5                      6                     50   

   (Column2, SubColumn2)  
0                     20  
1                     40  
2                     60  
